# What can we learn from this FASTA file?
### Developing some simple bioinformatic functions

Firstly, let's get the sequence out of the file. It pretty simply, but there are some things to watch out for, like headers and newlines. I like to double check my sequence length with UniProt before proceeding to be safe.

In [3]:
def get_sequence(infile):
    input_sequence = open(infile).readlines()[1:] # reads from after header
    clean_sequence = []
    for line in input_sequence:
        clean_line = line.replace("\n", "")
        clean_sequence.append(clean_line)
    sequence = "".join(clean_sequence) # joins strings. Can also be done by declaring and empty string and appending it 
    return sequence

faah2 = get_sequence('faah2.fasta')
print(faah2)

MAPSFTARIQLFLLRALGFLIGLVGRAALVLGGPKFASKTPRPVTEPLLLLSGMQLAKLIRQRKVKCIDVVQAYINRIKDVNPMINGIVKYRFEEAMKEAHAVDQKLAEKQEDEATLENKWPFLGVPLTVKEAFQLQGMPNSSGLMNRRDAIAKTDATVVALLKGAGAIPLGITNCSELCMWYESSNKIYGRSNNPYDLQHIVGGSSGGEGCTLAAACSVIGVGSDIGGSIRMPAFFNGIFGHKPSPGVVPNKGQFPLAVGAQELFLCTGPMCRYAEDLAPMLKVMAGPGIKRLKLDTKVHLKDLKFYWMEHDGGSFLMSKVDQDLIMTQKKVVVHLETILGASVQHVKLKKMKYSFQLWIAMMSAKGHDGKEPVKFVDLLGDHGKHVSPLWELIKWCLGLSVYTIPSIGLALLEEKLRYSNEKYQKFKAVEESLRKELVDMLGDDGVFLYPSHPTVAPKHHVPLTRPFNFAYTGVFSALGLPVTQCPLGLNAKGLPLGIQVVAGPFNDHLTLAVAQYLEKTFGGWVCPGKF


### What can we learn about its composition?
We'll start by taking a look at the amino acid counts

In [4]:
def residue_counts(sequence, percent = False):
    aa_residues = [
    'Q','W','E','R','T',
    'Y','I','P','A','S',
    'D','F','G','H','K',
    'L','C','V','N','M' ]
    # iterate through residues and increment counter 
    total_length = 0
    for residue in aa_residues:
        count = sequence.count(residue)
        total_length += count
        if percent == False: # check variable arguments
            print(residue, "count is", count)
        else:
            percentage = round(count/len(sequence) * 100, 2)
            print(residue, "count is", str(percentage) + "%")
    # Let's make sure we counted all the residues;
    # this will kick an error message if we didn't
    assert total_length == len(sequence)
residue_counts(faah2)


Q count is 19
W count is 7
E count is 25
R count is 17
T count is 20
Y count is 14
I count is 25
P count is 32
A count is 41
S count is 27
D count is 21
F count is 24
G count is 51
H count is 14
K count is 44
L count is 65
C count is 10
V count is 42
N count is 16
M count is 18


I've also included an argument to return the values as percents. I set it with the default value of False, so we don't have to remeber to give it a value if we won't be using it

In [5]:
residue_counts(faah2, percent = True)

Q count is 3.57%
W count is 1.32%
E count is 4.7%
R count is 3.2%
T count is 3.76%
Y count is 2.63%
I count is 4.7%
P count is 6.02%
A count is 7.71%
S count is 5.08%
D count is 3.95%
F count is 4.51%
G count is 9.59%
H count is 2.63%
K count is 8.27%
L count is 12.22%
C count is 1.88%
V count is 7.89%
N count is 3.01%
M count is 3.38%


Now let's throw in some chemistry and see what we can come up with. This version of the function will allow the user to choose between hydrophobic and hydrophilic residues, and to control the number of places after the decimal to round to.

In [9]:
def solubility(sequence, hydrophiles = True, round_to = 2):
    if hydrophiles == True:
        residues = ["D", "E", "R", "K"] # hydrophiles
    if hydrophiles == False:
        residues = ['L','A','F','Y','W','I', 'M','V'] # hydrophobes
    residue_count = 0
    for aa in residues:
        number_of_residues = sequence.count(aa)
        residue_count += number_of_residues
    hydrophilicity_percentage = residue_count / len(sequence) * 100
    return round(hydrophilicity_percentage, round_to)

sol_test_philes = solubility(faah2)
print('Percent Hydrophiles:', sol_test_philes)

# show using optional arguments 
sol_test_phobes = solubility(faah2, hydrophiles = False, round_to = 3)
print('Percent Hydrophobes:', sol_test_phobes)

Percent Hydrophiles: 20.11
Percent Hydrophobes: 44.361


### A good prototype, but...

What is neat about this version of the function is it lets the user call the same function to find counts for high or low solubility. The drawback is that the residue lists are hardcoded into the funciton. You might see a different list from lab to lab or project to project depending on the criteria or cutoff they use in determining hydrophobicity, etc. Let's give them control over the residues to count with a more generic function combining some of the tools in the previous two. It also makes out code more concise.

In [11]:
def display_residue_counts(sequence, aa_residues = ['D','E','R','K'],
                           percent = False, round_to = 2):
    target_count = 0
    for residue in aa_residues:
        count = sequence.count(residue)
        target_count += count
        if percent == False:
            print(residue, "count is", count)
        else:
            percentage = round(count/len(sequence) * 100, round_to)
            print(residue, "count is", str(percentage) + "%")
        return target_count #for qc purposes

# With longer lists, it is nice to declare your list outside the function to keep it from getting hideous
my_residue_list = ['D','E','K','R','H']
soluble_res = display_residue_counts(faah2, aa_residues = my_residue_list)

assert display_residue_counts('THADRYAN') == 1

D count is 21
D count is 1


In [12]:
my_phobic_list = ['L', 'A','F','Y','W','I','M','C','V']
insoluble_res = display_residue_counts(faah2, aa_residues = my_phobic_list,
                                       percent = True, round_to = 3)

L count is 12.218%


# What next?
We now have a handy function for displaying data. If we want to return a value for calculations, we can use a similar but distinct function. Yes, we could make displaying and\ argument in the other one, but to keep it easy to read and maintain, let's compartmentalize

In [14]:
def residue_percent(sequence, aa_residues = ['D','E','R','K'], round_to = 2):
    total_count = 0
    for residue in aa_residues:
        count = sequence.count(residue)
        total_count += count
    percentage = round(total_count/len(sequence) * 100, round_to)
    return percentage
    
assert residue_percent('DERK') == 100

 # Getting more practical: which kmers to choose for sythesis?
Let's say we've been funded to synthesize five 15mers from this protein for our research project. We aren't sure which ones to choose, are even how many 15mers there are in it. How can we find good targets? First off, let's see what we are working with.

In [15]:
def kmers(kmer_length, sequence):
    # initialize 'window' to slide along sequence 
    start = 0
    kmer_end = kmer_length
    sequence_end = len(sequence)
    # store results in list
    kmer_pool = []
    # while the window isn't sliding off the end, keep going by one
    while start + kmer_length <= sequence_end:
        kmer = sequence[start:kmer_end]
        kmer_pool.append(kmer)
        start += 1
        kmer_end += 1
    return kmer_pool
samples = kmers(15, faah2)
# the length of the list will tell us how many there are 
print(len(samples))
# let's peek are the first ten to see we're on track
print(samples[:10])

518
['MAPSFTARIQLFLLR', 'APSFTARIQLFLLRA', 'PSFTARIQLFLLRAL', 'SFTARIQLFLLRALG', 'FTARIQLFLLRALGF', 'TARIQLFLLRALGFL', 'ARIQLFLLRALGFLI', 'RIQLFLLRALGFLIG', 'IQLFLLRALGFLIGL', 'QLFLLRALGFLIGLV']


### QC'ing ourselves
Before we send any requisition orders over to the wetlab, we should check that we are reading these right. We can fact check outselves like this...

In [9]:
# We know the length of the protein
length = len(faah2)
print(length)

# So we know it is one 532mer long, right?
assert len(kmers(532, faah2)) == 1

# And that would also mean that it can be thought of as 532 1mers
assert len(kmers(1, faah2)) == 532

532


### Narrowing it down...
So knowing there are 518 kmers isn't super helpful in practice. Let's assume we want to find targets that would be fairly easy to sythesize and have a decent chance of being accsesible in the actual 3D structure of the protein. We'll use charge/solubility as a heuristic here, as those parts are easier to get into solution during synthesis and more likely to be oriented outward in the tertiary structure. 


In [16]:
sample_kmers = kmers(15, faah2)

kmer_dict = {}
for sample in sample_kmers:
    number = residue_percent(sample)
    kmer_dict[sample] = number

# sort our results from highest to lowest        
ranked_kmers = sorted(kmer_dict, key = kmer_dict.get, reverse = True)
for item in ranked_kmers:
    print(item, kmer_dict[item])        

EESLRKELVDMLGDD 53.33
KLAEKQEDEATLENK 53.33
DQKLAEKQEDEATLE 53.33
EKYQKFKAVEESLRK 53.33
KRLKLDTKVHLKDLK 53.33
EEKLRYSNEKYQKFK 53.33
KYQKFKAVEESLRKE 53.33
KFKAVEESLRKELVD 53.33
LLEEKLRYSNEKYQK 46.67
HAVDQKLAEKQEDEA 46.67
RFEEAMKEAHAVDQK 46.67
NEKYQKFKAVEESLR 46.67
YQKFKAVEESLRKEL 46.67
QKFKAVEESLRKELV 46.67
RLKLDTKVHLKDLKF 46.67
VEESLRKELVDMLGD 46.67
LEEKLRYSNEKYQKF 46.67
KYRFEEAMKEAHAVD 46.67
ESLRKELVDMLGDDG 46.67
FKAVEESLRKELVDM 46.67
EKLRYSNEKYQKFKA 46.67
QKLAEKQEDEATLEN 46.67
AEKQEDEATLENKWP 46.67
IKRLKLDTKVHLKDL 46.67
KEAHAVDQKLAEKQE 46.67
EKQEDEATLENKWPF 46.67
AVDQKLAEKQEDEAT 46.67
RYSNEKYQKFKAVEE 46.67
LAEKQEDEATLENKW 46.67
GIKRLKLDTKVHLKD 46.67
KAVEESLRKELVDML 46.67
AHAVDQKLAEKQEDE 46.67
VDQKLAEKQEDEATL 46.67
EAHAVDQKLAEKQED 46.67
LRYSNEKYQKFKAVE 40.0
LRKELVDMLGDDGVF 40.0
KGHDGKEPVKFVDLL 40.0
KVHLKDLKFYWMEHD 40.0
KLDTKVHLKDLKFYW 40.0
GIVKYRFEEAMKEAH 40.0
INGIVKYRFEEAMKE 40.0
HDGKEPVKFVDLLGD 40.0
IVKYRFEEAMKEAHA 40.0
NGIVKYRFEEAMKEA 40.0
SNEKYQKFKAVEESL 40.0
AKLIRQRKVKCIDVV 40.0
